In [1]:
# Reference from NLP 📝 GloVe, BERT, TF-IDF, LSTM... 📝 Explained. (n.d.). Retrieved April 18, 2024, 
#from kaggle.com website: https://www.kaggle.com/code/andreshg/nlp-glove-bert-tf-idf-lstm-explained#7.-LSTM

import numpy as np
import matplotlib.pyplot as plt
import os
import random
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train_df = pd.read_json('train.jsonl', lines=True)
X_train = train_df['string']
y_train = train_df['label']

test_df = pd.read_json('test.jsonl', lines=True)
X_test = test_df['string']
y_test = test_df['label']

train_df

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0000,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0000,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0000,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0000,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0000,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8238,explicit,50.0,,28.0,"Importantly, the results of Pascalis et al. (2...",background,0.7350,6f68ccd37718366c40ae6aeedf0b935bf560b215,60ed4bdabf92b2fbd6162dbd8979888cccca55d7,True,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,6f68ccd37718366c40ae6aeedf0b935bf560b215>60ed4...,15,NaN,NaN
8239,explicit,182.0,DISCUSSION,179.0,"As suggested by Nguena et al, there is a need ...",background,0.7508,f2a1c1704f9587c94ed95bc98179dc499e933f5e,574e659da7f6c62c07bfaaacd1f31d65bd75524c,True,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,f2a1c1704f9587c94ed95bc98179dc499e933f5e>574e6...,1,NaN,NaN
8240,explicit,120.0,DISCUSSION,108.0,Skeletal muscle is also a primary site of dise...,background,1.0000,18c97ea2ff60c110cc2a523e0fdf729608cbb083,fc13b9c3dfcc121013edaa12fa8ce7842aaed21a,False,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,18c97ea2ff60c110cc2a523e0fdf729608cbb083>fc13b...,8,NaN,NaN
8241,explicit,221.0,,185.0,ACTIVATION OF TRANSCRIPTION FACTORS Roles for ...,method,NaN,4ec9b89857c0b27e8a4bd3745b7358f387773527,81affdba19e38e2b17cf7b9e93792cc2028cf21d,True,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,4ec9b89857c0b27e8a4bd3745b7358f387773527>81aff...,0,NaN,NaN


In [3]:
df=train_df[['string','label']]
df_test=test_df[['string','label']]

In [4]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
df['string_clean'] = df['string'].apply(clean_text)
df_test['string_clean'] = df_test['string'].apply(clean_text)
df_test.head()

<ipython-input-4-1d019467fce9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_clean'] = df['string'].apply(clean_text)
<ipython-input-4-1d019467fce9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['string_clean'] = df_test['string'].apply(clean_text)


,string,label,string_clean
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel as well as upc coarray fortran and...
1,"In addition, the result of the present study s...",result,in addition the result of the present study su...
2,Several instruments that more specifically add...,background,several instruments that more specifically add...
3,Organotypic hippocampal slice cultures\nInterf...,method,organotypic hippocampal slice culturesinterfac...
4,Activated PBMC are the basis of the standard P...,background,activated pbmc are the basis of the standard p...


In [7]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

df['string_clean'] = df['string_clean'].apply(remove_stopwords)
df_test['string_clean'] = df_test['string_clean'].apply(remove_stopwords)
df_test.head()

<ipython-input-7-30cb1b1ca881>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_clean'] = df['string_clean'].apply(remove_stopwords)
<ipython-input-7-30cb1b1ca881>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['string_clean'] = df_test['string_clean'].apply(remove_stopwords)


,string,label,string_clean
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel well upc coarray fortran titanium ...
1,"In addition, the result of the present study s...",result,addition result present study supports previou...
2,Several instruments that more specifically add...,background,several instruments specifically address patie...
3,Organotypic hippocampal slice cultures\nInterf...,method,organotypic hippocampal slice culturesinterfac...
4,Activated PBMC are the basis of the standard P...,background,activated pbmc basis standard pbmc blast assay...


In [6]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text
df['string_clean'] = df['string_clean'].apply(stemm_text)
df_test['string_clean'] = df_test['string_clean'].apply(stemm_text)
df_test.head()

<ipython-input-8-c00e444b63c9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_clean'] = df['string_clean'].apply(stemm_text)
<ipython-input-8-c00e444b63c9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['string_clean'] = df_test['string_clean'].apply(stemm_text)


,string,label,string_clean
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel well upc coarray fortran titanium ...
1,"In addition, the result of the present study s...",result,addit result present studi support previous st...
2,Several instruments that more specifically add...,background,sever instrument specif address patientreport ...
3,Organotypic hippocampal slice cultures\nInterf...,method,organotyp hippocamp slice culturesinterfac hip...
4,Activated PBMC are the basis of the standard P...,background,activ pbmc basi standard pbmc blast assay neu...


In [9]:
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))

    return text
df['string_clean'] = df['string_clean'].apply(preprocess_data)
df_test['string_clean'] = df_test['string_clean'].apply(preprocess_data)
df_test.head()

<ipython-input-9-7a7880891bf5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_clean'] = df['string_clean'].apply(preprocess_data)
<ipython-input-9-7a7880891bf5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['string_clean'] = df_test['string_clean'].apply(preprocess_data)


,string,label,string_clean
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel well upc coarray fortran titanium ...
1,"In addition, the result of the present study s...",result,addit result present studi support previous st...
2,Several instruments that more specifically add...,background,sever instrument specif address patientreport ...
3,Organotypic hippocampal slice cultures\nInterf...,method,organotyp hippocamp slice culturesinterfac hip...
4,Activated PBMC are the basis of the standard P...,background,activ pbmc basi standard pbmc blast assay neu...


In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['label'])
le.fit(df_test['label'])
df['label_encoded'] = le.transform(df['label'])
df_test['label_encoded'] = le.transform(df_test['label'])
df_test.head()

<ipython-input-10-58c42f99e4ed>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_encoded'] = le.transform(df['label'])
<ipython-input-10-58c42f99e4ed>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['label_encoded'] = le.transform(df_test['label'])


,string,label,string_clean,label_encoded
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,chapel well upc coarray fortran titanium ...,0
1,"In addition, the result of the present study s...",result,addit result present studi support previous st...,2
2,Several instruments that more specifically add...,background,sever instrument specif address patientreport ...,0
3,Organotypic hippocampal slice cultures\nInterf...,method,organotyp hippocamp slice culturesinterfac hip...,1
4,Activated PBMC are the basis of the standard P...,background,activ pbmc basi standard pbmc blast assay neu...,0


In [12]:
#x_train=df['string_clean']
#y_train = df['label_encoded']
#x_test=df_test['string_clean']
#y_test=df_test['label_encoded']

In [50]:
x_train = pd.concat([df['string_clean'], df_test['string_clean']], axis=0).reset_index(drop=True)
y_train = pd.concat([df['label_encoded'], df_test['label_encoded']], axis=0).reset_index(drop=True)
#x_train.loc[:300]

0      howev frataxin interact fes cluster biosynthes...
1      studi hickey et al  spike sampl field point ph...
2      drug also reduc catecholamin secret therebi re...
3      cluster lowli aggress close kin king  viblanc ...
4      ophthalm symptom rare manifest intracrani arac...
                             ...                        
296    fetal rat lung fibroblast isol  spraguedawley ...
297    static singl assign ssa  standard program repr...
298    although backreact hl hkl explicit includ mode...
299    adopt workflow refer model  describ major func...
300                   numer experi set suggest  β  − η  
Name: string_clean, Length: 301, dtype: object

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
# instantiate the vectorizer
#vect2 = CountVectorizer()
vect.fit(x_train)
#vect2.fit(x_test)

CountVectorizer()

In [41]:
# Use the trained to create a document-term matrix from train and test sets
x_train_dtm = vect.transform(x_train)

#x_test_dtm = vect2.transform(x_test)

In [42]:



texts = pd.concat([df['string_clean'], df_test['string_clean']], axis=0).reset_index(drop=True)
target = pd.concat([df['label_encoded'], df_test['label_encoded']], axis=0).reset_index(drop=True)


In [33]:
# Calculate the length of our vocabulary
from tensorflow.keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

27693

In [43]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
def embed(corpus):
    return word_tokenizer.texts_to_sequences(corpus)

ltrain = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(ltrain))

train_padded_sentences = tf.keras.preprocessing.sequence.pad_sequences(
    embed(texts),
    length_long_sentence,
    padding='post'
)

train_padded_sentences

array([[   63,  3648,   104, ...,     0,     0,     0],
       [    4,  4886,     1, ...,     0,     0,     0],
       [  455,     9,   103, ...,     0,     0,     0],
       ...,
       [   21,    83,  1874, ...,     0,     0,     0],
       [   67,  4555,  7695, ...,     0,     0,     0],
       [ 2936, 27692, 10445, ...,     0,     0,     0]], dtype=int32)

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('glove.6B.100d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

# embeddings_dictionary

In [45]:
# Glove dictionary. Others will be initialized to 0.

embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.044606  , -0.61925   , -1.14530003, ..., -1.36409998,
         0.35633001, -0.47716001],
       [-0.41431999, -0.46675   , -0.91295999, ..., -0.84961998,
         0.62581003, -0.63766998],
       ...,
       [-0.14822   , -0.03576   , -0.013765  , ...,  0.18188   ,
        -0.44797999, -0.25701001],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
import numpy as np


embedding_matrix = np.zeros((vocab_length, embedding_dim))

# Iterate over each word and its assigned index
for word, index in word_tokenizer.word_index.items():
    # Retrieve the embedding vector for the current word
    embedding_vector = embeddings_dictionary.get(word)

    # Check if the embedding vector exists
    if embedding_vector is not None:
        # Assign the embedding vector to the corresponding index in the matrix
        embedding_matrix[index] = embedding_vector


for i in range(vocab_length):
    if np.all(embedding_matrix[i] == 0):
        # Assigning a random vector from a normal distribution
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Print or return the embedding matrix
print(embedding_matrix)


In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop

def glove_lstm():
    model = Sequential()

    model.add(Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights = [embedding_matrix],
        input_length=length_long_sentence
    ))

    model.add(Bidirectional(LSTM(
        length_long_sentence,
        return_sequences = True,
        recurrent_dropout=0.2
    )))

    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = glove_lstm()
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 278, 100)          2769300   
                                                                 
 bidirectional_11 (Bidirect  (None, 278, 556)          842896    
 ional)                                                          
                                                                 
 global_max_pooling1d_11 (G  (None, 556)               0         
 lobalMaxPooling1D)                                              
                                                                 
 batch_normalization_11 (Ba  (None, 556)               2224      
 tchNormalization)                                               
                                                                 
 dropout_33 (Dropout)        (None, 556)               0         
                                                     

In [51]:
train_padded_sentences

array([[   63,  3648,   104, ...,     0,     0,     0],
       [    4,  4886,     1, ...,     0,     0,     0],
       [  455,     9,   103, ...,     0,     0,     0],
       ...,
       [   21,    83,  1874, ...,     0,     0,     0],
       [   67,  4555,  7695, ...,     0,     0,     0],
       [ 2936, 27692, 10445, ...,     0,     0,     0]], dtype=int32)

In [52]:
target

0        0
1        0
2        0
3        0
4        0
        ..
10099    2
10100    0
10101    1
10102    0
10103    1
Name: label_encoded, Length: 10104, dtype: int64

In [54]:
'''X_train, X_test, y_train, y_test = train_test_split(
    train_padded_sentences,
    target,
    test_size=0.25
)'''
X_train=train_padded_sentences[:8243]
X_test=train_padded_sentences[8243:]
y_train=target[:8243]
y_test=target[8243:]

In [61]:

model = glove_lstm()
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    'model.h5',
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True
)
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    verbose = 1,
    patience = 5,
    min_lr = 0.001
)
history = model.fit(
    X_train,
    y_train,
    epochs = 2,
    batch_size = 32,
    validation_data = (X_test, y_test),
    verbose = 1,
    callbacks = [reduce_lr, checkpoint]
)

Epoch 1/2
258/258 [==============================] - ETA: 0s - loss: -209.4879 - accuracy: 0.5094
Epoch 1: val_loss improved from inf to -877.60229, saving model to model.h5
258/258 [==============================] - 1607s 6s/step - loss: -209.4879 - accuracy: 0.5094 - val_loss: -877.6023 - val_accuracy: 0.5083 - lr: 0.0010
Epoch 2/2


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


258/258 [==============================] - ETA: 0s - loss: -12687.9189 - accuracy: 0.5352
Epoch 2: val_loss improved from -877.60229 to -26808.29883, saving model to model.h5
258/258 [==============================] - 1508s 6s/step - loss: -12687.9189 - accuracy: 0.5352 - val_loss: -26808.2988 - val_accuracy: 0.4616 - lr: 0.0010
